In [1]:
import tensorflow as tf
from tensorflow import keras
from keras.models import load_model
import numpy as np
import pyaudio
import time
import librosa
import wave
import queue
import IPython.display as ipd
from IPython.display import Audio
import soundfile as sf
import os
import threading
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

In [2]:
# Định nghĩa các hằng số
RATE = 16000
CHUNK = 1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
n_times_1s = int(RATE / CHUNK * 1)  # Number of iterations to record 1 second
n_times_145s = int(RATE*1.5/ CHUNK)  # Number of iterations to record 1 second
predict_mode = 0
p = pyaudio.PyAudio()

In [3]:
def features_extractor_1s(file_name):
    audio, sample_rate = librosa.load(file_name,sr=16000, mono=True)
    frame_length = int(0.025 * sample_rate)
    hop_length = int(0.015 * sample_rate)
    n_fft = 2 ** int(np.ceil(np.log2(frame_length)))
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    desired_size = 66
    mfccs = np.zeros((13,desired_size))
    mfccs[:, :min(desired_size, mfcc.shape[1])] = mfcc[:, :min(desired_size, mfcc.shape[1])]
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    mfccs_features = np.transpose(mfccs_features)

    return mfccs_features

In [4]:
# Hàm trích xuất đặc trưng MFCC
def features_extractor_145s(file_name):
    audio, sample_rate = librosa.load(file_name,sr=16000, mono=True)
    frame_length = int(0.025 * sample_rate)
    hop_length = int(0.015 * sample_rate)
    n_fft = 2 ** int(np.ceil(np.log2(frame_length)))
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_fft=n_fft, hop_length=hop_length, n_mfcc=13)
    desired_size = 100
    mfccs = np.zeros((13,desired_size))
    mfccs[:, :min(desired_size, mfcc.shape[1])] = mfcc[:, :min(desired_size, mfcc.shape[1])]
    delta_mfccs = librosa.feature.delta(mfccs)
    delta2_mfccs = librosa.feature.delta(mfccs, order=2)
    mfccs_features = np.concatenate((mfccs, delta_mfccs, delta2_mfccs))
    mfccs_features = np.transpose(mfccs_features)

    return mfccs_features

In [5]:
def mic_check_1s(file_name):
    mfcc_processed = features_extractor_1s(filename)
    prediction = model_1s.predict(np.expand_dims(mfcc_processed, axis=0))
    if prediction[:, 0] > 0.99:
        print(f"Wake Word Detected for ({i})")
        print("Confidence:", prediction[:, 1])
        display(Audio(filename))
    return 0

In [6]:
# Hàm dự đoán số 
def mic_check_145s(file_name):
    global predict_mode
    mfccs_features = features_extractor_145s(file_name)
    # print(mfccs_features.shape())
    # data = mfccs_features
    data = np.stack(mfccs_features)
    data = np.reshape(data,(1,100,39))
    pre_test = model_145s.predict(data, verbose=0)
    print(pre_test)
    s=0
    result = 0
    for i in pre_test[0]:
        if i > 9e-1:
            result = i
            break
        s=s+1
    
    if s == 0:
        print("Predict: Batdieuhoa")
        display(Audio(get_file_path('Response','Batdieuhoa.wav'), rate=16000, autoplay=True))
    elif s == 1:
        print("Predict: Tatdieuhoa")
        display(Audio(get_file_path('Response','Tatdieuhoa.wav'), rate=16000, autoplay=True))
    elif s == 2:
        print("Predict: Tang1do")
        display(Audio(get_file_path('Response','Tang1do.wav'),  rate=16000,autoplay=True))
    # elif s == 3:
    #     print("Predict: Tang2do")
    # elif s == 4:
    #     print("Predict: Tang3do")
    elif s == 3:
        print("Predict: Giam1do")
        display(Audio(get_file_path('Response','Giam1do.wav'), rate=16000, autoplay=True))
    # elif s == 5:
    #     print("Predict: Giam2do")
    # elif s == 7:
    #     print("Predict: Giam3do")
    elif s == 4:
        print("Predict: Bat26do")
        display(Audio(get_file_path('Response','Bat26do.wav'), rate=16000, autoplay=True))
    elif s == 5:
        print("Predict: Silent")
    else:
        print("Unrecognizable")
    if s < 5:
        predict_mode = 0
        time.sleep(2)
    print('Reliability: %.2f %%' %(result*100.00))
    display(Audio(file_name))
    
#         print('Reliability: %.2f %%' %(result*100.00))
#         display(Audio(file_name))
    

In [7]:
def get_file_path(folder,file_name):
    # Lấy đường dẫn đến thư mục làm việc hiện tại trong Jupyter
    current_directory = os.getcwd()

    # Đường dẫn đến thư mục chứa tệp mô hình "H5"
    h5_folder_path = os.path.join(current_directory, folder)

    # Đường dẫn đến tệp mô hình được chỉ định
    file_path = os.path.join(h5_folder_path, file_name)

    return file_path

In [8]:
# Hàm của luồng 1
def record_audio_1s(file_number, stream):
    print(f"Recording audio for file {file_number}...")
    global predict_mode
    frames = []

    for _ in range(n_times_1s):
        # Lấy một khung dữ liệu
        data = stream.read(CHUNK)
        frames.append(data)

    # Lưu file âm thanh vào file wav
    filename =  "wakeup_{}".format(file_number)+".wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    return filename
        

In [9]:
# Hàm của luồng 1
def record_audio_145s(file_number, stream):
    print(f"Recording audio for file {file_number}...")
    frames = []

    for _ in range(n_times_145s):
        # Lấy một khung dữ liệu
        data = stream.read(CHUNK)
        frames.append(data)

    # Lưu file âm thanh vào file wav
    filename =  "speech_{}".format(file_number)+".wav"
    with wave.open(filename, 'wb') as wf:
        wf.setnchannels(CHANNELS)
        wf.setsampwidth(p.get_sample_size(FORMAT))
        wf.setframerate(RATE)
        wf.writeframes(b''.join(frames))
    return filename

In [10]:
# Hàm của luồng 1
def record_audio():
    print("Recording audio...")
    global predict_mode
    #Khởi tạo Audio
    stream = p.open(format=FORMAT,
                    channels=1,
                    rate=RATE,
                    input=True,
                    frames_per_buffer=CHUNK)
    for i in range(500):
        filename = record_audio_1s(i, stream)
        mfcc_processed = features_extractor_1s(filename)
        data = np.stack(mfcc_processed)
        data = np.reshape(data,(66,39,1))
        prediction = model_1s.predict(np.expand_dims(data, axis=0))
        print(prediction)
        if prediction[:, 0] > prediction[:, 1]:
            print(f"Wake Word Detected for ({i})")
            print("Confidence:", prediction[:, 1])
            display(Audio(get_file_path('Response','Hello.wav'), rate=16000, autoplay=True))
            display(Audio(filename))
            predict_mode = 0
            while (predict_mode < 10):
                speech = record_audio_145s(predict_mode, stream)
                mic_check_145s(speech)
                predict_mode += 1
                !del {speech}
            display(Audio(get_file_path('Response','Bye.wav'), rate=16000, autoplay=True))
        !del {filename}
        
        
    
    # Đóng stream và thoát khỏi PyAudio
    stream.stop_stream()
    stream.close()
    p.terminate()

    
      
    

In [11]:
model_145s = load_model(get_file_path("H5","Train_100_True_AIS_150_29_LSTM.h5"), compile=False)
# Au_AN_Rain
model_1s = load_model(get_file_path("WuW_H5","Train_66_WuW_True_AIS_1s.h5"), compile=False)

In [ ]:
record_audio()

Recording audio...
Recording audio for file 0...
1/1 [==============================] - 0s 173ms/step
[[1.4739885e-11 1.0000000e+00]]
Recording audio for file 1...
1/1 [==============================] - 0s 32ms/step
[[1.578419e-17 1.000000e+00]]
Recording audio for file 2...
1/1 [==============================] - 0s 33ms/step
[[2.8936533e-21 1.0000000e+00]]
Recording audio for file 3...
1/1 [==============================] - 0s 30ms/step
[[7.7327263e-19 1.0000000e+00]]
Recording audio for file 4...
1/1 [==============================] - 0s 54ms/step
[[7.247285e-19 1.000000e+00]]
Recording audio for file 5...
1/1 [==============================] - 0s 16ms/step
[[4.7760417e-11 1.0000000e+00]]
Recording audio for file 6...
1/1 [==============================] - 0s 31ms/step
[[5.672909e-16 1.000000e+00]]
Recording audio for file 7...
1/1 [==============================] - 0s 31ms/step
[[0.9989886  0.00101315]]
Wake Word Detected for (7)
Confidence: [0.00101315]


Recording audio for file 0...
[[1.6131427e-14 6.0010760e-15 7.3713517e-14 1.2303883e-13 1.4262220e-17
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 1...
[[3.8817635e-08 8.8508095e-05 2.0867684e-03 4.1048269e-04 9.5433110e-09
  9.9741411e-01]]
Predict: Silent
Reliability: 99.74 %


Recording audio for file 2...
[[2.7574389e-13 4.2536433e-13 1.0677562e-15 8.8789924e-17 1.6124926e-19
  1.0000000e+00]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 3...
[[1.8981750e-03 9.9809259e-01 9.2325754e-06 1.9679821e-13 6.5519115e-19
  2.3242984e-11]]
Predict: Tatdieuhoa


...

Recording audio for file 2...
[[1.7095516e-06 1.1005414e-08 7.1959214e-12 1.1628896e-11 1.5210286e-10
  9.9999833e-01]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 3...
[[1.5567676e-11 7.6142292e-09 4.2839358e-07 5.5066057e-06 1.7106080e-13
  9.9999404e-01]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 4...
[[2.8007252e-03 4.4187278e-02 1.4351183e-04 1.6107626e-04 7.9477477e-06
  9.5269942e-01]]
Predict: Silent
Reliability: 95.27 %


Recording audio for file 5...
[[4.5024171e-10 1.7617598e-14 8.5626628e-15 3.7647885e-10 8.4380005e-08
  9.9999988e-01]]
Predict: Silent
Reliability: 100.00 %


Recording audio for file 6...
